# **NOTEBOOK D'UTILISATION DE CERTAINES FONCTIONALITEES PERSO**

In [5]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
from datetime import datetime
from collections import Counter
from geoalchemy2 import Geometry,WKTElement
import geopandas as gp
import pandas as pd
import numpy as np
import shutil, subprocess

from Outils import creer_graph, plus_proche_voisin,nb_noeud_unique_troncon_continu,verif_index, gp_changer_nom_geom
from Decorateurs import concat_df

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 50)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#TELECHARGER DES DONNEES depuis leserveur geomatique et les passer dans un fichier
@concat_df
def creer_dept(fichier):
    return gp.read_file(fichier)

liste_f=[fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_0{d}.SHP' for d in  ['16','17','86','79','19','23','87','33','24','40','64','47']]
df_tot=creer_dept(liste_f)

In [ ]:
#trsnfere des données dans une bdd
with ct.ConnexionBdd('local_otv_station_gti') as c :
    for d in ['17','86','79','19','23','87','33','24','40','64','47'] :
        fichier=fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_0{d}.SHP'
        ct.ogr2ogr_shp2pg(c.connstringOgr,fichier,schema='plamade', table='troncon_route_bdt17_l',SRID='2154',
                          geotype='MULTILINESTRINGZ', dims=3, creationMode='-append -update',encodageClient='UTF-8')

In [6]:
with ct.ConnexionBdd('local_otv') as c :
    fichier=r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\IGN\Bdtopo_BdxMet_ed18_l.shp'
    ct.ogr2ogr_shp2pg(c.connstringOgr,fichier,schema='linearisation_bm', table='Bdtopo_BdxMet_ed18_l',SRID='2154',
                          geotype='LINESTRINGZ', dims=3, creationMode='',encodageClient='UTF-8')

debut import fichier D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\IGN\Bdtopo_BdxMet_ed18_l.shp avec shape2pg à 10:55:40 
 avec commande ogr2ogr  -f "postgreSQL" --config PG_USE_COPY YES -a_srs "EPSG:2154"  -nlt LINESTRINGZ -dim 3 -lco "SCHEMA=linearisation_bm" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=otv user=postgres password=postgres port=5432" D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\IGN\Bdtopo_BdxMet_ed18_l.shp -nln linearisation_bm.Bdtopo_BdxMet_ed18_l 
Fait


In [5]:
with ct.ConnexionBdd('local_otv_station_gti') as c :
    fichier=fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_016.SHP'
    ct.Ogr2ogr_pg2shp(c.connstringOgr,r'D:\temp\otv\2018\livrables_OTR_2018\sigena_d_2014_2018_p.shp',
                     "select * from livrable.sigena_d_2014_2018_p")

debut export fichier D:\temp\otv\2018\livrables_OTR_2018\sigena_d_2014_2018_p.shp 
 a 14:04:30 
 avec commande ogr2ogr -f "ESRI shapefile" D:\temp\otv\2018\livrables_OTR_2018\sigena_d_2014_2018_p.shp PG:"host=localhost dbname=statio_gti_otv user=postgres password=postgres port=5432"  -sql "select * from livrable.sigena_d_2014_2018_p" 
Fait


In [33]:
with ct.ConnexionBdd('local_otv_station_gti') as c :
    ct.ogr2ogr_dbf2pg(c.connstringOgr,r'D:\temp\PlaMADE\base_nationale\structure_PlaMADE38\N_ROUTIER_ROUTE_038.dbf',
                      schema='plamade', table='n_routier_route_france',creationMode='-update -append',encodageClient='UTF-8',fieldmap='0,-1,-1,-1,4,-1,-1,-1,-1,-1,-1,-1,-1,-1')

debut import fichier D:\temp\PlaMADE\base_nationale\structure_PlaMADE38\N_ROUTIER_ROUTE_038.dbf avec shape2pg à 01:47:05 
 avec commande ogr2ogr -update -append -f "postgreSQL" --config PG_USE_COPY YES -lco "SCHEMA=plamade" PG:"host=localhost dbname=statio_gti_otv user=postgres password=postgres port=5432" D:\temp\PlaMADE\base_nationale\structure_PlaMADE38\N_ROUTIER_ROUTE_038.dbf -nln plamade.n_routier_route_france -fieldmap 0,-1,-1,-1,4,-1,-1,-1,-1,-1,-1,-1,-1,-1
Fait


In [ ]:
    """connexion=c.connstringOgr.replace(' ','\"',1)
    creationMode='-update -append'
    #creationMode=''
    schema='plamade'
    table='n_routier_route_france'
    encodage='SET PGCLIENTENCODING='+'UTF-8'
    redirection_gdaldata="cd C:\Program Files\GDAL\gdal-data"
    fichier=r'D:\temp\PlaMADE\base_nationale\structure_PlaMADE38\N_ROUTIER_ROUTE_038.dbf'
    fieldmap='0,-1,-1,-1,4,-1,-1,-1,-1,-1,-1,-1,-1,-1'
    sql=f'select idroute from {schema}.{table}'
    cmd=f'ogr2ogr {creationMode} -f "postgreSQL" --config PG_USE_COPY YES -lco "SCHEMA={schema}" {connexion}\" {fichier} -nln {schema}.{table}'
    commande=redirection_gdaldata+" && "+encodage+" && "+cmd
    print(f"debut import fichier {fichier} avec shape2pg à {datetime.now().time().isoformat(timespec='seconds')} \n avec commande {cmd}")
    subprocess.call(commande,shell=True)
    print('Fait')
    """